# RECOMMENDATION SYSTEM USING YELP DATASET

## BASIC MODEL

## INSTALLING LIBRARIES


In [172]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install scipy


## IMPORTING LIBRARIES

In [173]:
import pandas as pd
import numpy as np
from google.colab import drive
import json


## MOUNTING GOOGLE DRIVE

In [174]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


LOADING BUSINESS.JSON

In [175]:
business_df=pd.read_json("drive/MyDrive/YELP/yelp_academic_dataset_business.json",lines=True)

In [176]:
business_df["name"].nunique()

114117

In [177]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [178]:
business_df=business_df[business_df["is_open"]==1]

In [179]:
business_df.shape

(119698, 14)

## LOADING REVIEWS.JSON INTO DATASET

In [180]:
reviews_df=pd.read_json(open("drive/MyDrive/YELP/yelp_academic_dataset_review.json","r"),lines=True,chunksize=100000)

# FILTERING THE LARGE REVIEW.JSON DATAFRAME

In [181]:
count=0
dataframe=pd.DataFrame()
for review in reviews_df:
  indices=np.where((review["useful"]>0) & (review["cool"]>0) & (review["funny"]==0))
  dataframe=pd.concat([dataframe,review.iloc[indices]])



In [182]:

(dataframe["useful"]).sort_values(ascending=False).value_counts(normalize=True)

1      0.461334
2      0.255658
3      0.129969
4      0.065334
5      0.034531
         ...   
128    0.000001
129    0.000001
131    0.000001
134    0.000001
404    0.000001
Name: useful, Length: 138, dtype: float64

In [183]:
(dataframe["cool"]).sort_values(ascending=False).value_counts(normalize=True)

1      0.713521
2      0.192509
3      0.057132
4      0.019801
5      0.007668
         ...   
98     0.000001
99     0.000001
108    0.000001
109    0.000001
404    0.000001
Name: cool, Length: 106, dtype: float64

In [184]:
indices_helpful=np.where((dataframe["useful"]>=5) & (dataframe["cool"]>1))
dataframe_helpful=dataframe.iloc[indices_helpful]
dataframe_helpful.reset_index(inplace=True,drop=True)

In [185]:
dataframe_helpful.shape

(39351, 9)

In [186]:
dataframe_reduced=dataframe_helpful[["user_id","business_id","stars"]]


In [187]:
dataframe_reduced.head()

,user_id,business_id,stars
0,uAu772KpSkb-tPFgZmU-lA,2GYg3liJ9-m6Z67L_4_BRQ,5
1,uzy_KYIZx65cp7Yh8_seeQ,ZuM1vcZ_ObCjCCGweYzItg,5
2,qPORuflcC27780nm7s2UUg,V9XlikTxq0My4gE8LULsjw,1
3,3V_eOrneBc8Y0HmxKys6Yg,VcTddMHnMZeKibhRFWuqVw,5
4,-DMJL_fcLZCvlqzMm97giA,ZVyDNLVFF8K4zcnhoKNSbA,5


In [188]:
dataframe_reduced.shape

(39351, 3)

In [189]:
dfexp=dataframe_reduced.groupby(["user_id","business_id"],as_index=False)["stars"].mean()


In [190]:
dfexp["count"] = dfexp["business_id"].map(dfexp["business_id"].value_counts())

In [191]:
df_final=dfexp.loc[dfexp["count"]>2]

In [192]:
df_final.shape

(16992, 4)

In [193]:
df_final.head()

,user_id,business_id,stars,count
4,--u09WAjW741FdfkJXxNmg,tkY5BJVXxUmP4jsKavkf2g,5.0,5
8,-0W641mhJoejArA1X19T2w,IRh7j-5UmzQsq0pOHzpGkA,4.5,7
11,-1K2a9BI9qjlma21HmY0HA,8n_BlTxfALO08FWdb6Tnbg,5.0,3
14,-1MF2tosrw2WcCxeVNk81Q,h8qIDmSY6CgqvKMCjzzWeA,5.0,12
17,-1WbN1Qd-opw8u3uEqs2Kg,NyqYpVVIbEQvz6zKE-Fyew,5.0,13


In [194]:
table=df_final.pivot(index="business_id",columns="user_id",values="stars").fillna(0)

In [195]:
table.shape

(3341, 11103)

In [196]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [197]:
id_to_name=dict.fromkeys(business_df["business_id"])
for i in range(len(business_df)):
  id_to_name[business_df.iloc[i]["business_id"]]=business_df.iloc[i]["name"]


In [198]:
common=list(set(table.index).difference(set(business_df["business_id"])))
table.drop(common,inplace=True)

In [199]:
table.shape

(2772, 11103)

In [200]:


model=NearestNeighbors(algorithm="brute",metric="cosine",n_neighbors=6)
scores=csr_matrix(table.values)
model.fit(scores)

d_business={}
for business in table.index:
  recommendations=model.kneighbors([table.loc[business,:].values],return_distance=False)
  recommendations_names=[id_to_name[table.index[recommendation]] for recommendation in recommendations[0][1:]]
  d_business[id_to_name[business]]=recommendations_names

item_to_item=pd.DataFrame(d_business)

item_to_item.head()

,IndeBlue Modern Indian Food & Spirits,Spice 28,Dose's Pet Vets,Art's Skillet,Balance Chiropractic Health Center & Spa,Rick's Reef,Festiva,Grand Ole Opry,Rollin Fatties,RIZE - Indianapolis,...,Hot Green Tea Nails & Spa,Lithia Reno Subaru,Arch Of Reno,Amelia’s,The Bakeshop on 20th,PokeDoke,Cup Cafe,Free Tours By Foot,The Urban Juicer,Paul Mitchell The School Reno
0,Isot Mediterranean Cuisine,La Teresita Cafe,Pizzava,Lincoln Square Pancake House,Zama,American Social,3 Sisters Café,KD Electric,KD Electric,Seasons 52,...,Manatee Viewing Center,Hunter Creek Waterfall Trail,Bertha Miranda's Mexican Food Restaurant and C...,Love Handle,Race Street Cafe,Golden Gems,Antoine's Restaurant,Buddakan,Pinewood Social,Reno Smiles Dentistry and Orthodontics
1,1 Tippling Place,Tuk Tuk Thai Street Food,Black Rabbit Meads,Calvin Fletcher's Coffee Company,Christmas Village In Philadelphia,Crown & Bull,Three Carrots Fountain Square,Craft Ale House,Craft Ale House,Indianapolis City Market,...,CMX CinéBistro Hyde Park,Too Soul Tea Co.,Crawfish Asian Cuisine,Sushi Boss,Trek Bicycle - Philadelphia,Mayo Ketchup By Plantain Girl,Baja Cafe on Campbell,Taste Restaurant At Lucky Lou Seafood and Dimsum,Italia Pizza & Pasta,Brickies Tavern
2,Varga Bar,Craft Ale House,Fly High Trampoline Park Reno,Liter House,NECTAR,Cask Social Kitchen,Strange Bird,Taste Restaurant At Lucky Lou Seafood and Dimsum,Taste Restaurant At Lucky Lou Seafood and Dimsum,Daredevil Hall,...,Taste Restaurant At Lucky Lou Seafood and Dimsum,Jack's Cafe,"Galaxy Theatres Legends IMAX, Sparks",Garfield Park Conservatory and Sunken Gardens,Hollyhedge Estate,The Gramophone,The Bombay Club,Manatee Viewing Center,Tennessee Brew Works,emPIEnada
3,Queen Village Animal Hospital,CMX CinéBistro Hyde Park,Andelin Family Farm,Cleo’s Bodega & Cafe,None Such Farm Market,Sea Critters Cafe,Eiteljorg Museum of American Indians & Western...,Sher-e-Punjab,Sher-e-Punjab,Old National Centre,...,KD Electric,Kaffe Crepe,999 Pho,Seasons 52,City Fitness - Graduate Hospital,Ted Drewes,Tucson International Airport - TUS,CMX CinéBistro Hyde Park,Craft Ale House,Great Basin Brewing - Reno
4,Alamodak Restaurant and Hookah Lounge,KD Electric,The Eddy,Woody's Library Restaurant,Shiroi Hana,Fred Howard Park,Doughnuts & Dragons,Manatee Viewing Center,Manatee Viewing Center,Nesso,...,Sher-e-Punjab,Ijji Sushi,BJ's Nevada Barbecue,Root & Bone Indianapolis,Fantagio Spa and Body,2nd Shift Brewing,Prep & Pastry,KD Electric,Sun King Brewery,Village Center Farmers Market


## **NOTE** : THIS MODEL IS FIRST IMPLEMENTATION FOR THE RECOMMENDER SYSTEM. THE MODEL CAN BE FURTEHR IMPROVED WHEN BUSINESS FEATURES ARE CONSIDERED. THE BUSINESS FEATURES THAT CAN BE CONSIDERED ARE


## 1. LATITUDE
## 2. LONGITUDE
## 3. OVERALL BUSINESS RATING
## 4. REVIEW COUNT
## 5. PARKING FEATURES OF RESTAURANT
## 6. TAKEOUT IS AVAILABLE IN RESTAURANT OR NOT



## AFTER CREATING THE FEATURES FOR EACH DIFFERENT BUSINESS, DATA SHOULD BE CONVERTED INTO APPROPRIATE FORM TO FEED TO THE MODEL , THIS BECOMES INPUT TO THE MODEL AND THE OUTPUT OF THR MODEL ARE THE RATINGS BY THE USER. THERE ARE MULTIPLE TARGETS IN THIS CASE AND MULTITARGET LINEAR REGRESSION CAN BE USED.